In [1]:
import transformers
from transformers import DistilBertTokenizer,BertTokenizer
import pandas as pd
import nltk 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification,TFTrainingArguments
from transformers import TFTrainer

c:\Users\rvaib\.conda\envs\toxicsenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rvaib\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rvaib\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rvaib\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
train_df = pd.read_csv('data/train.csv')
# test_x =pd.read_csv('data/test.csv') 
# test_y =pd.read_csv('data/test_labels.csv')


In [4]:
# test_df = pd.merge(test_x,test_y,how='inner',on='id')
# test_df =test_df[test_df["toxic"]!=-1].reset_index(drop=True)
# test_df.head()

In [5]:
train_df =train_df.sample(n=10000,random_state=42)
# test_df =test_df.sample(n=2000,random_state=42)


In [6]:
stop_words =set(stopwords.words('english'))
lemmatizer =WordNetLemmatizer()

In [7]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens ]
    return ' '.join(tokens)

In [8]:
train_df['cleaned_text'] = train_df['comment_text'].apply(preprocess_text)



In [9]:
train_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned_text
119105,7ca72b5b9c688e9e,"Geez, are you forgetful! We've already discus...",0,0,0,0,0,0,"geez , forgetful ! 've already discussed marx ..."
131631,c03f72fd8f8bf54f,Carioca RFA \n\nThanks for your support on my ...,0,0,0,0,0,0,carioca rfa thanks support request adminship ....
125326,9e5b8e8fc1ff2e84,"""\n\n Birthday \n\nNo worries, It's what I do ...",0,0,0,0,0,0,"`` birthday worry , 's ; ) enjoy ur day|talk|e ``"
111256,5332799e706665a6,Pseudoscience category? \n\nI'm assuming that ...,0,0,0,0,0,0,pseudoscience category ? 'm assuming article p...
83590,dfa7d8f0b4366680,"(and if such phrase exists, it would be provid...",0,0,0,0,0,0,"( phrase exists , would provided search engine..."
...,...,...,...,...,...,...,...,...,...
25725,4412b93f983379a4,"Sam, get real! No-one's gunner believe all thi...",0,0,0,0,0,0,"sam , get real ! no-one 's gunner believe stuf..."
46449,7c1bd75a71704cc2,Get a life \n\nwhy dont you go get a life and ...,1,0,0,0,1,0,get life dont go get life stop editing article...
108844,45eed1057ea7a582,"Nothing a sock puppet(and trust me, I know wha...",0,0,0,0,0,0,"nothing sock puppet ( trust , know mean , sure..."
142353,f9718cf5cbdd9a25,"""\nThe lead section is a summary of the main p...",0,0,0,0,0,0,"`` lead section summary main point , mention i..."


In [10]:
X =list(train_df['cleaned_text'])
y = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()

In [24]:
y

[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0,

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =train_test_split(X,y, test_size=0.20,random_state=42)

In [12]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [13]:
train_encoding =tokenizer(X_train,truncation=True,padding=True)
test_encoding =tokenizer(X_test,truncation=True,padding=True)

In [37]:
# y_train_one_hot = tf.one_hot(y_train, 6)
# y_test_one_hot = tf.one_hot(y_test, 6)

In [45]:
train_dataset = tf.data.Dataset.from_tensor_slices((
  dict(train_encoding),y_train  
))
test_dataset = tf.data.Dataset.from_tensor_slices((
  dict(test_encoding),y_test
))


In [46]:
training_args =TFTrainingArguments(
    output_dir='C:/Users/rvaib/OneDrive/Desktop/nlp_with_NLTK/nlp/content/output1',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content',
    logging_steps=10,
    eval_steps=1
)


In [49]:
with training_args.strategy.scope():
    model =TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=6)
    model.config.num_labels = 6  # Assuming you have 6 labels
    model.classifier.activation = tf.keras.activations.sigmoid
trainer =TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    
)  

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_179']
You should probably TRAIN this model on a down-stream task to be able to use 

In [50]:
trainer.train()


c:\Users\rvaib\.conda\envs\toxicsenv\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


ValueError: in user code:

    File "c:\Users\rvaib\.conda\envs\toxicsenv\lib\site-packages\transformers\trainer_tf.py", line 711, in distributed_training_steps  *
        self.args.strategy.run(self.apply_gradients, inputs)
    File "c:\Users\rvaib\.conda\envs\toxicsenv\lib\site-packages\transformers\trainer_tf.py", line 653, in apply_gradients  *
        gradients = self.training_step(features, labels, nb_instances_in_global_batch)
    File "c:\Users\rvaib\.conda\envs\toxicsenv\lib\site-packages\transformers\trainer_tf.py", line 636, in training_step  *
        per_example_loss, _ = self.run_model(features, labels, True)
    File "c:\Users\rvaib\.conda\envs\toxicsenv\lib\site-packages\transformers\trainer_tf.py", line 758, in run_model  *
        outputs = self.model(features, labels=labels, training=training)[:2]
    File "c:\Users\rvaib\.conda\envs\toxicsenv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) 